<a href="https://colab.research.google.com/github/taminororo/DiGress/blob/main/Graff_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ディレクトリ構造が訳分からなくなったから、仕切り直し

In [11]:
# Google Driveのマウントと、環境の初期化（リセット）
from google.colab import drive
import os
import shutil

# 1. Driveをマウント
drive.mount('/content/drive')

# 2. 作業ベースディレクトリ
base_dir = '/content/drive/MyDrive/DiGress_Project'

# 3. 既存のフォルダがあれば削除してリセット（入れ子構造を解消するため）
if os.path.exists(base_dir):
    print(f"既存の {base_dir} を削除してクリーンアップします...")
    shutil.rmtree(base_dir)

# 4. フォルダ作成と移動
os.makedirs(base_dir, exist_ok=True)
os.chdir(base_dir)

# 5. Git Clone (DiGressフォルダが作られます)
print("GitHubからリポジトリをクローンします...")
!git clone https://github.com/cvignac/DiGress.git

# 6. 正しいディレクトリへ移動
work_dir = os.path.join(base_dir, 'DiGress')
os.chdir(work_dir)
print(f"現在のディレクトリ: {os.getcwd()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
既存の /content/drive/MyDrive/DiGress_Project を削除してクリーンアップします...
GitHubからリポジトリをクローンします...
Cloning into 'DiGress'...
remote: Enumerating objects: 593, done.
remote: Counting objects: 100% (394/394), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 593 (delta 320), reused 232 (delta 232), pack-reused 199 (from 1)
Receiving objects: 100% (593/593), 4.58 MiB | 8.99 MiB/s, done.
Resolving deltas: 100% (379/379), done.
現在のディレクトリ: /content/drive/MyDrive/DiGress_Project/DiGress


In [12]:
# 必要なライブラリのインストール
print("ライブラリをインストールします...")
!pip install rdkit pyyaml hydra-core wandb pytorch-lightning torch_geometric

ライブラリをインストールします...


In [13]:
# Colabにgraph_toolsがないため、エラーが出ないようモックコードをsrc/main.pyに埋め込む
target_file = 'src/main.py'

# 読み込み
with open(target_file, 'r') as f:
    lines = f.readlines()

# すでに修正済みかチェック
if "sys.modules['graph_tool']" in "".join(lines):
    print("main.py はすでに修正済みです。")
else:
    # 偽装用コード
    mock_code = [
        "import sys\n",
        "from unittest.mock import MagicMock\n",
        "# graph_tool を偽装してエラーを回避\n",
        "sys.modules['graph_tool'] = MagicMock()\n",
        "sys.modules['graph_tool.all'] = MagicMock()\n",
        "\n"
    ]
    # 先頭に挿入
    new_lines = mock_code + lines
    with open(target_file, 'w') as f:
        f.writelines(new_lines)
    print("main.py に graph_tool 回避コードを追加しました。")

main.py に graph_tool 回避コードを追加しました。


In [14]:
import os
import shutil
from google.colab import files

# 修正対象のファイルと配置先ディレクトリ
target_file = 'src/datasets/qm9_dataset.py'
raw_dir = 'data/qm9/qm9_pyg/raw'

# 1. 保存先ディレクトリを作成
os.makedirs(raw_dir, exist_ok=True)

# 2. qm9_dataset.py の修正（ダウンロード回避 & Pickleエラー修正）
print("datasetファイルのコード修正を実行します...")
with open(target_file, 'r') as f:
    lines = f.readlines()

new_lines = []
fixed_url = False
fixed_pickle = False

for line in lines:
    # URLエラーの修正：download_url をコメントアウト（手動アップロードするためダウンロード処理は不要）
    if "download_url(self.raw_url2, self.raw_dir)" in line:
        new_lines.append(f"        # {line.strip()}  # Fixed by Colab Patch (Manual Upload)\n")
        fixed_url = True
    # Pickleエラーの修正：weights_only=False を追加
    elif "torch.load" in line and "weights_only" not in line:
        original_line = line.rstrip()
        if original_line.endswith(")"):
            new_lines.append(original_line[:-1] + ", weights_only=False)\n")
            fixed_pickle = True
        else:
            new_lines.append(line)
    else:
        new_lines.append(line)

with open(target_file, 'w') as f:
    f.writelines(new_lines)

print(f"qm9_dataset.py の修正完了: URL無効化={fixed_url}, Pickle修正={fixed_pickle}")

# 3. uncharacterized.txt のアップロード
target_filename = 'uncharacterized.txt'
dest_path = os.path.join(raw_dir, target_filename)

if not os.path.exists(dest_path):
    print(f"\n【重要】下のボタンから '{target_filename}' をアップロードしてください。")
    uploaded = files.upload()

    if target_filename in uploaded:
        # アップロードされたファイルを所定の場所に移動
        shutil.move(target_filename, dest_path)
        print(f"\n成功: {target_filename} を {raw_dir} に配置しました。")
    else:
        print(f"\n警告: {target_filename} がアップロードされませんでした。ファイル名を確認してください。")
else:
    print(f"\n確認: {target_filename} は既に存在するため、アップロードをスキップします。")

datasetファイルのコード修正を実行します...
qm9_dataset.py の修正完了: URL無効化=True, Pickle修正=True

【重要】下のボタンから 'uncharacterized.txt' をアップロードしてください。


Saving uncharacterized.txt to uncharacterized.txt

成功: uncharacterized.txt を data/qm9/qm9_pyg/raw に配置しました。


In [15]:
# CUDAのエラーの修正
import os

target_file = 'src/diffusion/extra_features.py'

print(f"{target_file} のインデントを再修正します...")

if not os.path.exists(target_file):
    print("エラー: ファイルが見つかりません。")
else:
    with open(target_file, 'r') as f:
        lines = f.readlines()

    new_lines = []

    # 修正対象のキーワード（先ほど書き換えた行）
    target_keyword = "torch.linalg.eigh(L.cpu())"

    for i, line in enumerate(lines):
        # 問題の行を見つけたら
        if target_keyword in line:
            # 1. すぐ上の行（空行以外）を探す
            prev_line = ""
            prev_indent = ""
            for j in range(len(new_lines) - 1, -1, -1):
                if new_lines[j].strip(): # 空行でなければ
                    prev_line = new_lines[j]
                    # 行頭の空白部分（インデント）を取得
                    prev_indent = prev_line[:len(prev_line) - len(prev_line.lstrip())]
                    break

            # 2. もし上の行が ':' で終わっていたら、インデントを深くする必要がある
            stripped_code = line.strip()
            if prev_line.strip().endswith(':'):
                # 上の行のインデント + スペース4つ
                new_indent = prev_indent + "    "
                fixed_line = new_indent + stripped_code + "\n"
            else:
                # そうでなければ上の行と同じインデント（念のため）
                fixed_line = prev_indent + stripped_code + "\n"

            new_lines.append(fixed_line)
            print(f"修正しました (Line {i+1}):")
            print(f"前: {line.rstrip()}")
            print(f"後: {fixed_line.rstrip()}")
        else:
            new_lines.append(line)

    # 上書き保存
    with open(target_file, 'w') as f:
        f.writelines(new_lines)

    print("-" * 20)
    print("再修正完了。実験コマンドを実行してください。")

src/diffusion/extra_features.py のインデントを再修正します...
--------------------
再修正完了。実験コマンドを実行してください。


In [16]:
# インデントのズレを修正
import os

target_file = 'src/diffusion/extra_features.py'

print(f"{target_file} を修正して、収束エラー(linalg error)を回避します...")

if not os.path.exists(target_file):
    print("エラー: ファイルが見つかりません。")
else:
    with open(target_file, 'r') as f:
        lines = f.readlines()

    new_lines = []
    fixed_count = 0

    # 修正前のコード（検索用）
    target_code = "torch.linalg.eigh(L)"
    # 修正後のコード（CPUで計算してGPUに戻す）
    replacement_code = "torch.linalg.eigh(L.cpu()); eigvals, eigvectors = eigvals.to(L.device), eigvectors.to(L.device)"

    for line in lines:
        if target_code in line and ".cpu()" not in line:
            # インデント（行頭の空白）を取得
            indent = line[:line.find(target_code)].replace("eigvals, eigvectors = ", "")

            # 書き換え（インデント + 変数代入 + 新しいコード）
            # 元の行が "eigvals, eigvectors = ..." で始まっていることを想定
            if "eigvals, eigvectors =" in line:
                 indent_only = line[:line.find("eigvals")]
                 new_line = indent_only + "eigvals, eigvectors = " + replacement_code + "\n"
                 new_lines.append(new_line)
                 fixed_count += 1
            else:
                 # 想定外の書き方の場合はそのままにする（安全策）
                 new_lines.append(line)
        else:
            new_lines.append(line)

    if fixed_count > 0:
        with open(target_file, 'w') as f:
            f.writelines(new_lines)
        print(f"完了: {fixed_count} 箇所を修正しました。")
    else:
        print("変更なし: すでに修正されているか、対象の行が見つかりませんでした。")

src/diffusion/extra_features.py を修正して、収束エラー(linalg error)を回避します...
完了: 1 箇所を修正しました。


In [17]:
import os

target_file = 'src/datasets/qm9_dataset.py'
raw_dir = 'data/qm9/qm9_pyg/raw'

print(f"{target_file} を修正します...")

if not os.path.exists(target_file):
    print("エラー: ファイルが見つかりません。")
else:
    with open(target_file, 'r') as f:
        lines = f.readlines()

    new_lines = []
    fixed_rename = False

    for line in lines:
        # os.rename で '3195404' を扱っている行を探してコメントアウト
        # 行が複数にまたがっている可能性もあるため、 '3195404' が含まれる os.rename 周辺を無効化します
        if "os.rename" in line and "'3195404'" in line:
            new_lines.append(f"        # {line.strip()}  # Fixed by Colab Patch (Skip Rename)\n")
            fixed_rename = True
        # 複数行にまたがる場合の2行目以降 (osp.join(self.raw_dir, 'uncharacterized.txt')) もケア
        # 直前の行がコメントアウトされた os.rename なら、この行もコメントアウトする簡単なロジック
        elif fixed_rename and "uncharacterized.txt" in line and line.strip().endswith(")"):
             new_lines.append(f"        # {line.strip()}  # Fixed by Colab Patch (Skip Rename part 2)\n")
             # フラグを戻す
             fixed_rename = False
        else:
            if fixed_rename: # renameが1行で完結していなかった場合、次の行も念のためコメントアウトしておく（安全策）
                 if "uncharacterized.txt" in line:
                     new_lines.append(f"        # {line.strip()} # Fixed by Colab Patch\n")
                 else:
                     new_lines.append(line)
                 fixed_rename = False
            else:
                new_lines.append(line)

    # 上書き保存
    with open(target_file, 'w') as f:
        f.writelines(new_lines)

    print("修正完了: ダウンロード後のリネーム処理を無効化しました。")

# 念のため uncharacterized.txt が存在するか再確認
unchar_path = os.path.join(raw_dir, 'uncharacterized.txt')
if os.path.exists(unchar_path):
    print(f"確認OK: {unchar_path} は存在します。")
else:
    # 万が一なければ空ファイル作成
    with open(unchar_path, 'w') as f:
        pass
    print(f"再作成: {unchar_path} を作成しました。")

src/datasets/qm9_dataset.py を修正します...
修正完了: ダウンロード後のリネーム処理を無効化しました。
確認OK: data/qm9/qm9_pyg/raw/uncharacterized.txt は存在します。


In [18]:
# データ分割ファイルmake_splits.pyの作成と実行
script_content = """
import os
import numpy as np
import pandas as pd

raw_dir = 'data/qm9/qm9_pyg/raw'
os.makedirs(raw_dir, exist_ok=True)

np.random.seed(42)
n_samples = 130831
all_indices = np.arange(n_samples)
np.random.shuffle(all_indices)

train_size = 100000
val_size = 20000 # 論文再現のため 20000 に設定
test_size = n_samples - train_size - val_size

train_idx = all_indices[:train_size]
val_idx = all_indices[train_size:train_size+val_size]
test_idx = all_indices[train_size+val_size:]

pd.DataFrame({'mol_id': train_idx}, index=train_idx).to_csv(os.path.join(raw_dir, 'train.csv'))
pd.DataFrame({'mol_id': val_idx}, index=val_idx).to_csv(os.path.join(raw_dir, 'val.csv'))
pd.DataFrame({'mol_id': test_idx}, index=test_idx).to_csv(os.path.join(raw_dir, 'test.csv'))
print(f"Splits created: Train={len(train_idx)}, Val={len(val_idx)}, Test={len(test_idx)}")
"""

with open("src/make_splits.py", "w") as f:
    f.write(script_content)

print("make_splits.py を実行します...")
!python src/make_splits.py

make_splits.py を実行します...
Splits created: Train=100000, Val=20000, Test=10831


# A100での学習のために打つべきコマンド

In [2]:
# 1. Google Driveをマウント
from google.colab import drive
import os

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. 作業ディレクトリへ移動
work_dir = '/content/drive/MyDrive/DiGress_Project/DiGress'
if os.path.exists(work_dir):
    os.chdir(work_dir)
    print(f"移動完了: {os.getcwd()}")
else:
    print("エラー: DiGressフォルダが見つかりません。Driveのパスを確認してください。")

# 3. ライブラリの再インストール（GPU変更で消えているため必須）
print("ライブラリをインストールしています...（数分かかります）")
!pip install rdkit pyyaml hydra-core wandb pytorch-lightning torch_geometric > /dev/null
print("インストール完了！")

# 4. main.py の graph_tool 対策が残っているか確認（念のため）
# もし消えていたら、ここで自動的に再注入します
try:
    with open('src/main.py', 'r') as f:
        content = f.read()
    if "sys.modules['graph_tool']" not in content:
        print("graph_tool 対策を再適用します...")
        mock_code = "import sys\nfrom unittest.mock import MagicMock\nsys.modules['graph_tool'] = MagicMock()\nsys.modules['graph_tool.all'] = MagicMock()\n\n"
        with open('src/main.py', 'w') as f:
            f.write(mock_code + content)
    else:
        print("graph_tool 対策は適用済みです。")
except FileNotFoundError:
    print("警告: src/main.py が見つかりません。ディレクトリが正しいか確認してください。")

移動完了: /content/drive/MyDrive/DiGress_Project/DiGress
ライブラリをインストールしています...（数分かかります）
インストール完了！
graph_tool 対策は適用済みです。


In [1]:
# WandBへのログイン
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: taminororo34 (taminororo34-nigata) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# 実験の実行
import os

# 現在のディレクトリを確認（.../DiGress_Project/DiGress であること）
print(f"実行ディレクトリ: {os.getcwd()}")

# PYTHONPATHを通す
os.environ['PYTHONPATH'] = os.getcwd()

# 実行コマンド
# dataset=qm9 : QM9データセット
# trainer.accelerator='gpu' : GPU使用
# trainer.max_epochs=1000 : 論文設定の1000エポック
!python src/main.py dataset=qm9 \
    +trainer.accelerator='gpu' \
    +trainer.devices=1 \
    +trainer.max_epochs=1000 \
    +trainer.strategy=auto

実行ディレクトリ: /content/drive/MyDrive/DiGress_Project/DiGress
Found rdkit, all good
Extracting /content/drive/MyDrive/DiGress_Project/DiGress/data/qm9/qm9_pyg/raw/qm9.zip
Processing...
100% 133885/133885 [01:26<00:00, 1544.67it/s]
Done!
Processing...
100% 133885/133885 [00:31<00:00, 4295.53it/s]
Done!
Processing...
100% 133885/133885 [00:24<00:00, 5458.29it/s]
Done!
Computing dataset smiles...
	Converting QM9 dataset to SMILES for remove_h=True...
Invalid molecule obtained.
	Converting QM9 dataset to SMILES 0.00%
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
Invalid molecule obtained.
